In [1]:
import cv2
import numpy as np 
import sklearn 
import scipy
import matplotlib.pyplot as plt
import argparse
import math

In [2]:
image = cv2.imread('302843992_485786496928626_6191234182030738228_n.png')

In [5]:
cv2.imshow('image',image)
cv2.waitKey(0)

-1

In [6]:
print("width: {} pixels".format(image.shape[1]))

width: 697 pixels


In [7]:
print("height: {} pixels".format(image.shape[0]))

height: 451 pixels


In [8]:
print("channels: {}".format(image.shape[2]))

channels: 3


In [ ]:
image[0,0]

array([254, 254, 254], dtype=uint8)

In [9]:
def rgb2gray(image):
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image[i][j]=0.3*image[i][j][0]+0.6*image[i][j][1]+0.1*image[i][j][2]
    return image

In [10]:
def rgb2graymax(image):
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image[i][j]=max(image[i][j][0],image[i][j][1],image[i][j][2])
    return image

In [11]:
cv2.imshow('image',rgb2gray(image))
cv2.waitKey(0)

-1

In [12]:
cv2.imshow('image',rgb2graymax(image))
cv2.waitKey(0)

-1

In [13]:
grayscale = rgb2graymax(image)
cv2.imwrite("grayscale.png",grayscale)

True

In [14]:
def split_image(image):
    zeros = np.zeros(image.shape[:2], dtype="uint8")
    red_image = np.zeros(image.shape[:3], dtype="uint8")
    green_image = np.zeros(image.shape[:3], dtype="uint8")
    blue_image = np.zeros(image.shape[:3], dtype="uint8")
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if sum(image[i,j] >= np.array([200, 200, 200])) == 3:
                blue_image[i,j] = image[i,j]
                green_image[i,j] = image[i,j]
                red_image[i,j] = image[i,j]
            else:
                blue_image[i,j] = np.array([image[i,j,0],0,0])
                green_image[i,j] = np.array([0,image[i,j,1],0])
                red_image[i,j] = np.array([0,0,image[i,j,2]])
    
    return red_image,green_image,blue_image

In [15]:
red,green,blue = split_image(image)

In [16]:
cv2.imshow("red",red)
cv2.imshow("green",green)
cv2.imshow("blue",blue)
cv2.waitKey(0)


-1

In [17]:
canvas = np.zeros((300, 300, 3), dtype = "uint8")

In [18]:
cv2.imshow("canvas",canvas)
cv2.waitKey(0)

-1

In [19]:
def in_polygon(x, y, polygon):
    # check if the point (x,y) is inside the polygon without cv2
    result = False
    for i in range(len(polygon)):
        j = (i+1)%len(polygon)
        if (polygon[i][1] > y) != (polygon[j][1] > y) and \
            x < (polygon[j][0] - polygon[i][0]) * (y - polygon[i][1]) / (polygon[j][1] - polygon[i][1]) + polygon[i][0]:
            result = not result
    return result


def draw(canvas,polygon,color,offset_x=0,offset_y=0):
    canvas = canvas.copy()
    polygon = np.array(polygon)
    polygon[:,0] += offset_x
    polygon[:,1] += offset_y
    
    # each pixel in canvas 
    for i in range(canvas.shape[0]):
        for j in range(canvas.shape[1]):
            # check if the pixel is inside the polygon
            if in_polygon(j,i,polygon):
                canvas[i,j] = color
            
    return canvas

In [20]:
triangle = [(0,0),(30,0),(0,40)]
canvas = draw(canvas,triangle,(204,204,0),5,5)
cv2.imshow("canvas",canvas)
cv2.waitKey(0)


-1

In [21]:
triangle = [(10,10),(35,10),(22,31)]
canvas = draw(canvas,triangle,(255,0,0),40,5)
cv2.imshow("canvas",canvas)
cv2.waitKey(0)

-1

In [22]:
square = [(0,0),(50,0),(50,50),(0,50)]
canvas = draw(canvas,square,(128,128,128),5,40)
cv2.imshow("canvas",canvas)
cv2.waitKey(0)

-1

In [23]:
rhombus = [(0,0),(20,0),(0,34),(-20,34)]
canvas = draw(canvas,rhombus,(0,255,0),75,50)
cv2.imshow("canvas",canvas)
cv2.waitKey(0)

-1

In [ ]:
M = np.float32([[1, 0, 25], [0, 1, 50]])

In [25]:
shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
cv2.imshow("Shifted Down and Right", shifted)
cv2.waitKey(0)

-1

In [35]:
def shift_image(image, dx, dy):
    image = np.roll(image, dy, axis=0)
    image = np.roll(image, dx, axis=1)
    if dy>0:
        image[:dy, :] = 0
    elif dy<0:
        image[dy:, :] = 0
    if dx>0:
        image[:, :dx] = 0
    elif dx<0:
        image[:, dx:] = 0
    return image

In [36]:
cv2.imshow("Shifted Down and Right", shift_image(image, 50, 100))
cv2.waitKey(0)

-1

In [8]:
M = np.float32([[1, 0, -50], [0, 1, -90]])
shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
cv2.imshow("Shifted Up and Left", shifted)
cv2.waitKey(0)

-1

In [38]:
cv2.imshow("Shifted Up and Left", shift_image(image, -50, -90))
cv2.waitKey(0)

-1

In [5]:
(h, w) = image.shape[:2]
center = (w // 2, h // 2)

In [41]:
def image_rotate(image, degrees):
    rads = math.radians(degrees)
    cx, cy = (image.shape[1]//2, image.shape[0]//2)
    height_rot_img = round(abs(image.shape[0]*math.sin(rads))) + round(abs(image.shape[1]*math.cos(rads)))
    width_rot_img = round(abs(image.shape[1]*math.cos(rads))) + round(abs(image.shape[0]*math.sin(rads)))
    rot_img = np.uint8(np.zeros((height_rot_img,width_rot_img,image.shape[2])))
    midx,midy = (width_rot_img//2, height_rot_img//2)
    for i in range(rot_img.shape[0]):
        for j in range(rot_img.shape[1]):
            x= (i-midx)*math.cos(rads)+(j-midy)*math.sin(rads)
            y= -(i-midx)*math.sin(rads)+(j-midy)*math.cos(rads)
            x=round(x)+cy
            y=round(y)+cx
            if (x>=0 and y>=0 and x<image.shape[0] and  y<image.shape[1]):
                rot_img[i,j,:] = image[x,y,:]
    return rot_img 


In [46]:
cv2.imshow("Rotated by -90 Degrees", image_rotate(image, -90))
cv2.waitKey(0)

-1

In [45]:
M = cv2.getRotationMatrix2D(center, -90, 1.0)
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("Rotated by -90 Degrees", rotated)
cv2.waitKey(0)

-1

In [47]:
cv2.imshow("Rotated by 45 Degrees", image_rotate(image, 45))
cv2.waitKey(0)

-1

In [49]:
rotated = cv2.warpAffine(image, M, (w, h))
M = cv2.getRotationMatrix2D(center, 45, 1.0)
cv2.imshow("Rotated by 45 Degrees", rotated)
cv2.waitKey(0)

-1

In [66]:
def resize_img(image, resize_width, resize_height):   
    original_width, original_height, channel = image.shape
    
    red_channel   = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel  = image[:, :, 2]
    
    resized_image = np.zeros((resize_width, resize_height, channel), dtype=np.uint8)
    
    x_scale = original_width/resize_width
    y_scale = original_height/resize_height
    
    resize_idx = np.zeros((resize_width, resize_height))
    resize_index_x = np.ceil(np.arange(0, original_width, x_scale)).astype(int)
    resize_index_y = np.ceil(np.arange(0, original_height, y_scale)).astype(int)
    resize_index_x[np.where(resize_index_x == original_width)]  -= 1
    resize_index_y[np.where(resize_index_y == original_height)] -= 1

    resized_image[:, :, 0] = red_channel[resize_index_x, :][:, resize_index_y]
    resized_image[:, :, 1] = green_channel[resize_index_x, :][:, resize_index_y]
    resized_image[:, :, 2] = blue_channel[resize_index_x, :][:, resize_index_y]
    
    return resized_image

In [67]:
r = 150.0 / image.shape[1]
dim = (150, int(image.shape[0] * r))
cv2.imshow("Resized (Width)", resize_img(image,dim[0],dim[1]))
cv2.waitKey(0)

-1

In [7]:
r = 150.0 / image.shape[1]
dim = (150, int(image.shape[0] * r))

In [9]:
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
cv2.imshow("Resized (Width)", resized)
cv2.waitKey(0)

-1

In [69]:
r = 50.0 / image.shape[0]
dim = (int(image.shape[1] * r), 50)
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
cv2.imshow("Resized (Height)", resized)
cv2.waitKey(0)

-1

In [68]:
r = 50.0 / image.shape[0]
dim = (int(image.shape[1] * r), 50)
cv2.imshow("Resized (Height)", resize_img(image,dim[0],dim[1]))
cv2.waitKey(0)

-1

In [8]:
flipped = cv2.flip(image, 1)
cv2.imshow("Flipped Horizontally", flipped)
cv2.waitKey(0)

-1

In [3]:
def flip(image,flip):
    if flip >= 1:
        h_flip = image[:, ::-1]
        return h_flip
    elif flip == 0: 
        v_flip = image[::-1]
        return v_flip
    else:
        h_flip = image[:, ::-1]
        v_flip = h_flip[::-1]
        return v_flip
# flip horizontally
    

In [7]:
cv2.imshow("Flipped Horizontally", flip(image,1))
cv2.waitKey(0)

-1

In [3]:
def crop_img(image,width,heigt,x,y):
    cropped = image[x:x+width , y:y+heigt]
    return cropped


In [19]:
crop = image[100:200, 500:600]
cv2.imshow("T-Rex Face", crop)
cv2.waitKey(0)

-1

In [16]:
cv2.imshow("T-Rex Face", crop_img(image,100,100,100,500))
cv2.waitKey(0)

-1

In [71]:
M = np.ones(image.shape, dtype = "uint8") * 100
added = cv2.add(image, M)
cv2.imshow("Added", added)
cv2.waitKey(0)

-1

In [11]:
def image_add_matrix(image,filters):
    result = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):  
        for j in range(image.shape[1]):
            if float(image[i,j,0]) + float(filters[i,j,0]) >= 255:
                result[i,j,0] = 255
            else:
                result[i,j,0] = float(image[i,j,0]) + float(filters[i,j,0])
            if float(image[i,j,1]) + float(filters[i,j,1]) >= 255:
                result[i,j,1] = 255
            else:
                result[i,j,1] = float(image[i,j,1]) + float(filters[i,j,1])
            if float(image[i,j,2]) + float(filters[i,j,2]) >= 255:
                result[i,j,2] = 255
            else:
                result[i,j,2] = float(image[i,j,2]) + float(filters[i,j,2])
    return result

In [14]:
M = np.ones(image.shape, dtype = "uint8")*100
cv2.imshow("Added", image_add_matrix(image,M))
cv2.waitKey(0)

-1

In [18]:
def image_substract_matrix(image,filters):
    result = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):  
        for j in range(image.shape[1]):
            if float(image[i,j,0]) - float(filters[i,j,0]) <= 0:
                result[i,j,0] = 0
            else:
                result[i,j,0] = float(image[i,j,0]) - float(filters[i,j,0])
            if float(image[i,j,1]) - float(filters[i,j,1]) <= 0:
                result[i,j,1] = 0
            else:
                result[i,j,1] = float(image[i,j,1]) - float(filters[i,j,1])
            if float(image[i,j,2]) - float(filters[i,j,2]) <= 0:
                result[i,j,2] = 0
            else:
                result[i,j,2] = float(image[i,j,2]) - float(filters[i,j,2])
    return result

In [19]:
M = np.ones(image.shape, dtype = "uint8") * 100
cv2.imshow("Subtracted", image_substract_matrix(image,M))
cv2.waitKey(0)

-1

In [16]:
M = np.ones(image.shape, dtype = "uint8") * 100
subtracted = cv2.subtract(image, M)
cv2.imshow("Subtracted", subtracted)
cv2.waitKey(0)

-1